In [1]:
from torch.utils.data import DataLoader
import gurobipy as gp
import numpy as np
import pandas as pd
import torch
from tensordict import TensorDict
import matplotlib.pyplot as plt


from online.EnergyDataset import EnergyDataset

In [2]:
ds = EnergyDataset('../data/1_processed/energy.csv','../data/1_processed/price.csv', 48, 1, 'train')
battery_capacity =  ds.getBatteryCapacity().item()
battery_power = battery_capacity / 4

In [3]:
old_soe = 0.0
timeslot = range(0, 48, 1)
model = gp.Model("battery_optimization")
model.setParam('OutputFlag', 0)

Set parameter Username
Set parameter LicenseID to value 2650516
Academic license - for non-commercial use only - expires 2026-04-11


In [16]:
td_array = []
for i in range(47,17424,48):
    prosumption, price, time_feature = ds[i]
    prosumption_array = prosumption.numpy().flatten()[1:49]
    price_array = price.numpy().flatten()[1:49]

    price_forecast = torch.zeros(49,48)
    price_forecast[0] = price[1:49]
    for j in range(1,49):
        _, price, _ = ds[i+j]
        price_forecast[j] = price[1:49]

    # Create TensorDict
    td_next = TensorDict({
        'prosumption': prosumption[1:49].unsqueeze(-1),
        'price': price[1:49].unsqueeze(-1),
        'time_feature': time_feature[1:49],
        'price_forecast': price_forecast[0:48],
    }, batch_size=[48])

    td = TensorDict({
        'prosumption': prosumption[0:48].unsqueeze(-1),
        'price': price[0:48].unsqueeze(-1),
        'time_feature': time_feature[0:48],
        'price_forecast': price_forecast[0:48],
        'next': td_next,
    }, batch_size=[48])

    # Gurobi model
    cost = []

    solution_actions = []
    solution_soe = []

    action = model.addVars(timeslot, lb=-battery_power, ub=battery_power, vtype=gp.GRB.CONTINUOUS, name="action")
    soe = model.addVars(range(0, 49, 1), vtype=gp.GRB.CONTINUOUS, name="soe")
    # Initial SOE constraint
    model.addConstr(soe[0] == 0.0, "Initial_SOE")
            
    # Define SOE constraints over time
    for t in timeslot:
        model.addConstr(soe[t + 1] == soe[t] + action[t], f"SOE_Update_{t}")
        model.addConstr(soe[t + 1] >= 0, f"SOE_Lower_Bound_{t}")
        model.addConstr(soe[t + 1] <= battery_capacity, f"SOE_Upper_Bound_{t}")

    # Define net load and objective function
    cost = gp.quicksum(price_array[t] * (prosumption_array[t] + action[t]) for t in timeslot)
    model.setObjective(cost, gp.GRB.MINIMIZE)

    # Optimize model
    model.optimize()
    optimal_actions = [action[t].x for t in timeslot]
    cost_array = [-price_array[t] * (prosumption_array[t] + optimal_actions[t]) for t in timeslot]

    solution_actions.extend([action[t].x for t in timeslot])
    solution_soe.extend([soe[t].x for t in range(0, 49, 1)])

    td['action'] = torch.tensor(solution_actions).unsqueeze(-1)
    td['soe'] = torch.tensor(solution_soe[0:48]).unsqueeze(-1)
    td['next', 'soe'] = torch.tensor(solution_soe[1:49]).unsqueeze(-1)
    td['next', 'reward'] = torch.tensor(cost_array).unsqueeze(-1)
    td['observation'] = torch.cat((td['soe'], td['prosumption'], td['time_feature'], td['price'], td['price_forecast']), dim=-1)
    td['next', 'observation'] = torch.cat((td['next', 'soe'], td['next', 'prosumption'], td['next','time_feature'], td['next', 'price'], td['next', 'price_forecast']), dim=-1)
    td_array.append(td)

In [18]:
torch.save(td_array, '../outputs/battery_optimization_solution_1.pt')

In [21]:
torch.load('../outputs/battery_optimization_solution_1.pt', weights_only=False)[0]

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([48, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        next: TensorDict(
            fields={
                observation: Tensor(shape=torch.Size([48, 53]), device=cpu, dtype=torch.float64, is_shared=False),
                price: Tensor(shape=torch.Size([48, 1]), device=cpu, dtype=torch.float64, is_shared=False),
                price_forecast: Tensor(shape=torch.Size([48, 48]), device=cpu, dtype=torch.float32, is_shared=False),
                prosumption: Tensor(shape=torch.Size([48, 1]), device=cpu, dtype=torch.float64, is_shared=False),
                reward: Tensor(shape=torch.Size([48, 1]), device=cpu, dtype=torch.float64, is_shared=False),
                soe: Tensor(shape=torch.Size([48, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                time_feature: Tensor(shape=torch.Size([48, 2]), device=cpu, dtype=torch.float64, is_shared=False)},
            batch_size=torch.Size([48]),
